## Setup: library modules and helper functions

In [33]:
import numpy as np                        # for efficient data types
import csv                                # to take a peek at our CSV files
import os                                 
import glob                               # pathnames matching
import pandas as pd                       # data manipulation and analysis with parallelization 
import warnings                           # to get rid of markdown warnings               
import matplotlib.pyplot as plt           # to support plotting visuals
import seaborn as sns                     # visualization module
from shapely.geometry import Point, Polygon
import difflib                            # for comparing strings
import re                                 # for pattern matching 
from datetime import datetime, timedelta  # for manipulating date and time columns

# global settings
sns.set(style="white")
%matplotlib inline
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 100)

# functions and variables that we'll reuse 
def cols_mem_usage(*args: str) -> None:
    """Print dataframe column's memory usage in megabytes."""
    df_mem_usage = df.memory_usage(deep = True)
    for col_name in list(args):
        print(f"{col_name} total memory usage: {df_mem_usage[col_name]/1_000_000}MB")
        
"""
pandas is inadvertantly converting integers into floats (i.e. 1 -> 1.0) on csv reads, so
this is mainly to handle comparisons that should hold true like '1' == '1.0'  
while accounting for strings without digits
"""
def try_int(x: str) -> str:
    """Attempt to convert a string into an integer, and back into a string to remove decimals."""
    try:
        return str(int(x))
    except ValueError:
        return x
#hiding any warnings showed by interpretor
warnings.simplefilter("ignore")

### Data preview

Let's look at a single row in one of the csv files we've downloaded to see if we need to omit any irrelevant columns from or inquiry

In [34]:
def get_csv_peek(fp: str) -> pd.DataFrame:   
    """Look at csv's first record as a dataframe"""
    with open(fp) as csvfile:
        reader = csv.reader(csvfile)
        column_headers = next(reader) 
        first_row = next(reader)
        return pd.DataFrame(data = [first_row], columns = column_headers)

get_csv_peek(r"F:\Coursera\Projects\Cyclistic_case_study\Trip Data\csv_files\202012-divvy-tripdata.csv")

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week
0,70B6A9A437D4C30D,classic_bike,27-12-2020 12:44,27-12-2020 12:55,Aberdeen St & Jackson Blvd,13157,Desplaines St & Kinzie St,TA1306000003,41.87772613,-87.65478743,41.88871604,-87.64444785,member,00:10:37,1


### Checking Dataset size

In [35]:
def get_files_total_size() -> float:
    """Find the total dataset size by summing each individual file's size"""
    data_folder_path = "F:\Coursera\Projects\Cyclistic_case_study\Trip Data\csv_files\\"
    files = list(os.listdir(data_folder_path))
    total_size = 0
    for f in files:
        total_size += os.stat(data_folder_path + f).st_size
    
    return round(total_size/1_000_000, 2)

print(f"Total dataset size: {get_files_total_size()} Megabytes")

Total dataset size: 1018.79 Megabytes


## Processing Scope

* Research and apply the right data manipulation tools for processing and analysis
* Check for dirty data - for data that is outdated, duplicated, incomplete, inconsistent and/or inaccurate/incorrect
* Transform and clean the data to a point where it can be worked with effectively in analysis

## Choosing the right tools for the job

Given the scale size of the merged dataset, we will be utilizing Python 3.0 with several library modules that are necessary for data manipulation and visualization. We will primarily be relying on the Pandas software library for its efficient use of data structures and operations. In case we run into any performance issues, we will take advantage of a cloud cluster querying solution like DB2.

## Checking for dirty data

### Merging our dataset into a single dataframe

In [36]:
def get_merged_df(data_folder_path: str) -> pd.DataFrame:
    """Merge data files into a single dataframe."""
    files = list(os.listdir(data_folder_path))
    file_paths = [data_folder_path + f for f in files]
    #load all files
    df_per_file = (pd.read_csv(fp, 
                               low_memory = False, 
                               parse_dates = ['started_at', 'ended_at'],
                               infer_datetime_format = True,
                               memory_map = True)
                  for fp in file_paths)
    return pd.concat(df_per_file, ignore_index = True)

df = get_merged_df("F:\Coursera\Projects\Cyclistic_case_study\Trip Data\csv_files\\")

In [37]:
df['ride_length'] = (df['ended_at'] - df['started_at'])/np.timedelta64(1, 'm') # turn it into minutes

### Examining our meta data

Let's get a sense of our main dataset by looking at the memory usage, formatting and descriptive statistics of each of our fields, among other things. We do this to validate our data before diving into data processing. 

In [38]:
# get information from our dataframe (number of records, memory use and data types)
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5479096 entries, 0 to 5479095
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  ride_length         float64       
 14  day_of_week         int64         
dtypes: datetime64[ns](2), float64(5), int64(1), object(7)
memory usage: 2.8 GB


In [39]:
df.describe().apply(lambda s: s.apply('{0:.4f}'.format))

,start_lat,start_lng,end_lat,end_lng,ride_length,day_of_week
count,5479096.0000,5479096.0000,5474358.0000,5474358.0000,5479096.0000,5479096.0000
mean,41.9013,-87.6459,41.9016,-87.6461,20.2720,4.0997
std,0.0459,0.0284,0.0460,0.0286,4088.5168,2.0811
min,41.6400,-87.8400,41.3900,-88.9700,-421899.0000,1.0000
25%,41.8817,-87.6600,41.8819,-87.6600,7.0000,2.0000
50%,41.8990,-87.6418,41.8999,-87.6427,12.0000,4.0000
75%,41.9291,-87.6277,41.9295,-87.6278,22.0000,6.0000
max,42.0700,-87.5200,42.1681,-87.4900,436887.0000,7.0000


### Checking for duplicate values

Let's now look at the number of unique values under each field

In [40]:
# checking the number of unique values per column
print("Unique values per column")
for col in df.columns:
    print(f'{col}: {df[col].nunique()}')

Unique values per column
ride_id: 5479084
rideable_type: 3
started_at: 410046
ended_at: 411037
start_station_name: 845
start_station_id: 830
end_station_name: 839
end_station_id: 825
start_lat: 377723
start_lng: 358168
end_lat: 443631
end_lng: 403731
member_casual: 2
ride_length: 4541
day_of_week: 7


- rideable_type and member_casual could benefit from a more efficient data type since they both have a limited number of unique values
- ride_id, presumably the intended primary key of this table, clearly has duplicates since the number of unique values does not conform to the dataframe's size.
- there are more station ids than there are station names, which means that many station names are linked to multiple IDs

### Checking for entity participation constraints

Let's check if IDs can also associate with more than one station name.

In [41]:
"""Get a pandas series of stations sharing the same ID and their value counts"""
start_ids_sr = df.groupby(['start_station_id','start_station_name']).size().rename_axis(index={'start_station_id': 'id', 'start_station_name': 'station_name'})
end_ids_sr = df.groupby(['end_station_id','end_station_name']).size().rename_axis(index={'end_station_id': 'id', 'end_station_name': 'station_name'})

vc1 = start_ids_sr.reset_index()['id'].value_counts()
vc2 = end_ids_sr.reset_index()['id'].value_counts()

start_ids_with_many_names = vc1[vc1 > 1].index
end_ids_with_many_names = vc2[vc2 > 1].index

start_mask = start_ids_sr.index.get_level_values(0).isin(start_ids_with_many_names)
start_ids_sr[start_mask]
end_mask = end_ids_sr.index.get_level_values(0).isin(end_ids_with_many_names)
end_ids_sr[end_mask]

inconsistent_ids_series = pd.concat([start_ids_sr[start_mask], end_ids_sr[end_mask]], axis = 0)
print("Number of station IDs with more than one name associated with it:", len(inconsistent_ids_series))
inconsistent_ids_series.head(6)

Number of station IDs with more than one name associated with it: 62


id     station_name                                       
13074  Broadway & Wilson - Truman College Vaccination Site     9222
       Broadway & Wilson Ave                                    430
13099  Halsted St & 18th St                                    1646
       Halsted St & 18th St (Temp)                             1730
13300  DuSable Lake Shore Dr & Monroe St                      19846
       Lake Shore Dr & Monroe St                              25757
dtype: int64

Looks like IDs can can link to several station names due to the time series nature of our data set and the evolution of station names and locations from events like relocations, renovations and renamings. Based on the fact that there are no IDs with more than two station names, we'll assume that these instances are valid. We'll keep it in mind for later in case it affects our analysis in some way. 

## Transforming and cleaning the data

### Resolving duplicate data

In [42]:
mask = (df['ended_at'] - df['started_at']) < pd.Timedelta("0 days")
print("Number of rows with negative ride durations:", len(df.loc[mask]))

Number of rows with negative ride durations: 1572


Now we will remove the rows with negative ride durations

In [43]:
df = df.loc[~mask]

### Formatting adjustment: converting rideable_type and member_casual

In [44]:
# let's convert these low unique count columns into categories
df['rideable_type'] = df['rideable_type'].astype('category')
df['member_casual'] = df['member_casual'].astype('category')

### Cleaning station ids 
We'll convert each station id into their integer equivalent

Let's see how many station ids contain characters

In [45]:
# checking the number of non digit values in start_station_id and end_station_id
mask = (df['start_station_id'].astype(str).str.contains(pat = '[^0-9.]', regex = True)) & (df['start_station_id'].notnull())
print(f"unique values with characters in start_station_id: {df.loc[mask, 'start_station_id'].nunique()}")

mask = (df['end_station_id'].astype(str).str.contains(pat = '[^0-9.]', regex = True)) & (df['end_station_id'].notnull())
print(f"unique values with characters in end_station_id: {df.loc[mask, 'end_station_id'].nunique()}")

unique values with characters in start_station_id: 303
unique values with characters in end_station_id: 303


We'll group stations by their ids and check to see which stations are connected to more than one ID.

In [46]:
def get_inconsistent_stations_index() -> pd.Index:
    """Get a pandas index of stations IDs (start and end) that share multiple names"""
    start_station_ids_df = df.groupby(['start_station_name', 'start_station_id']).size()
    end_station_ids_df = df.groupby(['end_station_name', 'end_station_id']).size()
    vc1 = start_station_ids_df.reset_index()['start_station_name'].value_counts()
    vc2 = end_station_ids_df.reset_index()['end_station_name'].value_counts()
    starts = vc1[vc1 > 1].index
    ends = vc2[vc2 > 1].index
    return starts.union(ends)

inconsistent_stations = get_inconsistent_stations_index()

print("Number of inconsistent stations with more than one id:",len(inconsistent_stations))
print('Names of the inconsistent stations: '+inconsistent_stations)

Number of inconsistent stations with more than one id: 1
Index(['Names of the inconsistent stations: Loomis St & 89th St'], dtype='object')


We'll remove any rows that contain 'Loomis St & 89th St' as either a start or end point for accuracy of the analysis assuming it will not affect the stats significantly

In [47]:
mask = (df['start_station_name'] == 'Loomis St & 89th St') | (df['end_station_name'] == 'Loomis St & 89th St')
df = df.loc[~mask]

In [48]:
inconsistent_stations = get_inconsistent_stations_index()
print("Number of inconsistent stations with more than one id:",len(inconsistent_stations))

Number of inconsistent stations with more than one id: 0


All stations now have a single linked station ID

In [49]:
df.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    651270
start_station_id      651267
end_station_name      698583
end_station_id        698583
start_lat                  0
start_lng                  0
end_lat                 4672
end_lng                 4672
member_casual              0
ride_length                0
day_of_week                0
dtype: int64

Let's just drop rows that have any missing station names or station ids since we don't have any external data to treat these missing values

In [50]:
mask = (df['start_station_name'].isna()) | (df['start_station_id'].isna())
df = df.loc[~mask]
mask = (df['end_station_name'].isna()) | (df['end_station_id'].isna())
df = df.loc[~mask]

##### Adding a distance column which will store the great circle distance between start and end station 

In [51]:
def haversine(lat1, lon1, lat2, lon2):
    """
    haversine formula determining the great-circle distance between two points on a sphere in miles
    """
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return 3956 * 2 * np.arcsin(np.sqrt(a))

df['distance'] = haversine(df['start_lat'],df['start_lng'],df['end_lat'],df['end_lng'])

And output our dataframe to a csv file for further analysis in tableau

Deleting columns which we don't need to save space

In [52]:
df.drop(['start_station_id', 'end_station_id'], axis = 1)

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week,distance
0,70B6A9A437D4C30D,classic_bike,2020-12-27 12:44:00,2020-12-27 12:55:00,Aberdeen St & Jackson Blvd,Desplaines St & Kinzie St,41.877726,-87.654787,41.888716,-87.644448,member,11.0,1,0.926431
39,15F369FDAED4E8E3,electric_bike,2020-12-18 13:53:00,2020-12-18 14:01:00,Larrabee St & Armitage Ave,Wells St & Walton St,41.918112,-87.643799,41.900129,-87.634448,member,8.0,6,1.331365
50,0CFD61DFE00E6043,electric_bike,2020-12-28 17:10:00,2020-12-28 17:12:00,Kingsbury St & Kinzie St,Desplaines St & Kinzie St,41.889193,-87.638576,41.889099,-87.642479,member,2.0,2,0.200711
87,244CB936487039B7,docked_bike,2020-12-10 13:36:00,2020-12-10 14:37:00,Clark St & Leland Ave,Clark St & Leland Ave,41.967096,-87.667429,41.967096,-87.667429,casual,61.0,5,0.000000
88,B7AD5038F79637F9,classic_bike,2020-12-20 13:09:00,2020-12-20 13:15:00,Dearborn St & Monroe St,Kingsbury St & Kinzie St,41.881320,-87.629521,41.889177,-87.638506,member,6.0,1,0.712459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5479074,03F06D61B2215C92,electric_bike,2021-11-02 13:50:00,2021-11-02 13:55:00,Green St & Randolph St,Kingsbury St & Kinzie St,41.883510,-87.648568,41.889181,-87.638859,member,5.0,3,0.634306
5479079,B102144E454381D4,classic_bike,2021-11-03 17:28:00,2021-11-03 17:45:00,Elston Ave & Cortland St,Wells St & Walton St,41.916433,-87.666746,41.899930,-87.634430,casual,17.0,4,2.013921
5479082,29EB49CAC5219D45,docked_bike,2021-11-07 11:55:00,2021-11-07 12:05:00,Desplaines St & Randolph St,Aberdeen St & Jackson Blvd,41.884616,-87.644571,41.877726,-87.654787,casual,10.0,1,0.708598
5479084,F4B91264B3D8FA9F,electric_bike,2021-11-22 18:03:00,2021-11-22 18:06:00,Ogden Ave & Chicago Ave,Desplaines St & Kinzie St,41.896442,-87.653998,41.888691,-87.644465,member,3.0,2,0.725578


In [53]:
df.to_csv('F:\Coursera\Projects\Cyclistic_case_study\Trip Data\csv_files\processed_cyclistic_data.csv', index = False)

## Analysis scope
- Organize and format the data (including useful aggregate tables)
- Identify trends and relationships with the help of calculations, data aggregations, and relevant visuals
- Provide a summary of the analysis

## Aggregates & Formatting

To get a sense of how members and casuals behave differently, we'll apply some aggregates around a variety of columns.

In [54]:
df.member_casual.value_counts()

member    2497501
casual    2026927
Name: member_casual, dtype: int64

#### Average Ride Length (minutes)

In [55]:
df.groupby('member_casual')['ride_length'].describe()[['mean', 'max', 'min']]

,mean,max,min
member_casual,,,
casual,119.500320,436887.0,0.0
member,29.718932,44686.0,0.0


Here we can see that the maximum ride length is 436887 minutes for casual riders and 44686 minutes for members which isn't possible. So we will delete those rows which have a maximum ride length greater than 1440 mins(24 hours)

In [56]:
df.drop(df[df.ride_length > 1440].index, inplace=True)

In [57]:
df.groupby(['member_casual', 'day_of_week'])['ride_length'].describe()[['mean', 'max', 'min']]

mean     max  min
member_casual day_of_week                        
casual        1            32.251542  1435.0  0.0
              2            28.533688  1439.0  0.0
              3            25.495635  1436.0  0.0
              4            24.331571  1438.0  0.0
              5            24.037844  1436.0  0.0
              6            25.989964  1433.0  0.0
              7            30.343145  1440.0  0.0
member        1            15.245324  1351.0  0.0
              2            12.790419  1370.0  0.0
              3            12.430305  1262.0  0.0
              4            12.557002  1415.0  0.0
              5            12.459965  1406.0  0.0
              6            12.865318  1427.0  0.0
              7            14.840514  1370.0  0.0

#### Average Distance Length

In [58]:
df.groupby('member_casual')['distance'].mean()

member_casual
casual    1.356058
member    1.300270
Name: distance, dtype: float64

In [59]:
df.groupby('member_casual')['distance'].std()

member_casual
casual    1.229899
member    1.116109
Name: distance, dtype: float64

#### Busiest Days of the Week

In [60]:
df.groupby('member_casual')['day_of_week'].value_counts()

member_casual  day_of_week
casual         7              462997
               1              401503
               6              283474
               2              226254
               5              219347
               4              214817
               3              213629
member         4              387873
               3              386324
               5              362047
               6              355015
               7              353298
               2              341619
               1              310340
Name: day_of_week, dtype: int64

In [61]:
df.day_of_week.mode()

0    7
dtype: int64

Clearly we can see that casual riders use bikes the most on Saturdays and members use bikes the most on Wednesdays and as a whole riders use bikes the most on Saturdays. As we thought, casuals are spending the majority of their bike trips on weekends while members spread out their use more evenly throughout the week. 

## Identifying Trends and Relationships

### Both User Types:

- Scarcely ride Cyclistic bikes during the colder months
- Pick up and drop off their bikes at roughly the same distance

### Casuals:

- Spend the majority of their bike trips on weekends
- Consistently have longer ride lengths year round
- Favor docked bikes for lengthy bike trips
- Have a negligible presence during the winter months

### Members:

- Spread out their use more evenly throughout the week.
- Have steady average ride lengths year round
- Are active at all times of the day, more so in the morning and afternoon 
- Are more consistent with their bike trips year round

### Summary: Inconclusive

A. Casuals primarily use Cyclistic bikes for leisure

We make that assumption based on the fact that casuals:

- Bike twice as much on Saturdays and Sundays compared to any other day of the week
- Spend significantly longer on average on every bike trip, suggesting that they spend time in-between docking stations doing leisurely activities 
- Do not use Cyclistic bikes often enough to warrant paying for an annual membership

B. Members get more our of Cyclistic bikes by using them for leisure and commuting on a consistent basis

We've drawn that conclusion based on the fact that members:

- Rely on bikes consistently each week and year round, with no notable preference on a single day of the week
- Use Cyclistic bikes often during the rush hours on a typical work day
- Are motivated by the economics of an annual membership pass 